<a href="https://colab.research.google.com/github/DaniFarias/Previsao_de_Tensoes_TCC_FURG/blob/main/Previs%C3%A3o_de_Tens%C3%B5es_TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# Importa bibliotecas

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn

In [ ]:
# Importa do dataset
tensoes = pd.read_csv('TensaoTeste.csv', thousands=',', delimiter=';', encoding='latin1', na_values='n/a')
tensoes.head(3)

In [ ]:
#list(tensoes)
#tensoes.info()
#tensoes.describe()

In [4]:
# Drop coluna Tempo
tensoes.drop(columns=['Tempo'], inplace = True)
tensoes.head(3)

In [5]:
# Para garantir um resultado idêntico, iniciamos a geração randomica com a mesma semente (seed)
np.random.seed(42)

In [6]:
# Split dos dados de treino e teste
from sklearn.model_selection import train_test_split

tensoes_treino, tensoes_teste = train_test_split(tensoes, 
                                                 test_size=0.2, 
                                                 random_state=42)
print('tensoes_treino:', len(tensoes_treino))
print('tensoes_teste: ', len(tensoes_teste))

tensoes_treino: 171752
tensoes_teste:  42938


In [7]:
# Separar treino
X = tensoes_treino.drop('Tensao12', axis=1)
y = tensoes_treino['Tensao12'].copy()

In [ ]:
tensoes.sample(50)

# 1. Modelo Arvore de Decisão

In [8]:
# Criação e treinamento do modelo por Arvore de Decisão
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42, 
                                 max_depth=10)
tree_reg.fit(X,y)

DecisionTreeRegressor(max_depth=10, random_state=42)

In [ ]:
#print('Acurácia: {:.5%}'.format(tree_reg.score(X, y)))

In [12]:
# Usando a medida de desempenho RMSE
from sklearn.metrics import mean_squared_error

tensoes_predictions = tree_reg.predict(X)

tree_mse = mean_squared_error(y, tensoes_predictions)
tree_rmse = np.sqrt(tree_mse)
print("RMSE:",tree_rmse)

RMSE: 112.17645060558498


In [11]:
# Validação Cruzada para o modelo de Árvore de Decisão
from sklearn.model_selection import cross_val_score

tree_scores = cross_val_score(tree_reg, X, y
                          ,scoring='neg_mean_squared_error'
                          ,cv=5)
tree_scores = np.sqrt(-tree_scores)
print("Mean:  ", tree_scores.mean())
print("Std:   ", tree_scores.std())

Mean:   112.94750705900552
Std:    4.75655878868191


# 2. Floresta Aleatória

In [13]:
# Aqui um modelo de Floresta Aleatória -- demora um pouco mais que os anteriores
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=30
                                   ,random_state=42)
forest_reg.fit(X, y)

RandomForestRegressor(n_estimators=30, random_state=42)

In [14]:
# Usando a medida de desempenho RMSE
from sklearn.metrics import mean_squared_error

tensoes_predictions = forest_reg.predict(X)

forest_mse = mean_squared_error(y, tensoes_predictions)
forest_rmse = np.sqrt(forest_mse)
print("RMSE:", forest_rmse)

RMSE: 2.6200649689198405


In [22]:
# Validação cruzada para o modelo de Floresta Aleatória
from sklearn.model_selection import cross_val_score

scores = cross_val_score(forest_reg, X, y
                         ,scoring='neg_mean_squared_error'
                         ,cv=10)
scores = np.sqrt(-scores)
print("Mean:  ", scores.mean())
print("Std:   ", scores.std())

Mean:   6.583189203841551
Std:    1.0818084910929295


In [37]:
# Exibe resultados
#print("Scores:", scores)
print("Mean:  ", scores.mean())
print("Std:   ", scores.std())

Mean:   6.583189203841551
Std:    1.0818084910929295


In [17]:
# Separar teste
X_test = tensoes_teste.drop('Tensao12', axis=1)
y_test = tensoes_teste['Tensao12'].copy()

In [18]:
# Usando a medida de desempenho RMSE para dados de Teste
final_predictions = forest_reg.predict(X_test)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)
print("RMSE: ",(final_rmse))

Erro:  7.230025757633563


In [19]:
# Usando a medida de desempenho CV para dados de Teste
scores = cross_val_score(forest_reg, X, y, 
                         scoring='neg_mean_squared_error', cv=10)
scores = np.sqrt(-scores)

In [38]:
#print("Scores:", scores)
print("Mean:  ", scores.mean())
print("Std:   ", scores.std())

Mean:   6.583189203841551
Std:    1.0818084910929295


In [ ]:
#Classificação de Features
forest_reg.feature_importances_

for score, name in sorted(zip(forest_reg.feature_importances_, tensoes)):
    print('{:.2%}'.format(score), name)

In [27]:
#print('Acurácia: {:.5%}'.format(forest_reg.score(X_test, y_test)))

In [24]:
z = tensoes_predictions.tolist()

In [ ]:
# Comparação tensoes de teste e preditas
pd.options.display.float_format = '{:.2f}'.format
df = pd.DataFrame((zip(y, z)), columns = ['T_12', 'T_12_predict'])
df.sample(50)